In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset
from PIL import Image
import os



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def get_dataset(data_dir):
    transform = transforms.Compose([
            transforms.ToTensor()
        ])
    train_dataset = datasets.ImageNet(
        root=data_dir, train=True, download= True,  transform=transform,
    )
    test_dataset = datasets.ImageNet(
        root=data_dir, train=False, download= True,  transform=transform,
    )
    full_dataset = ConcatDataset([train_dataset, test_dataset])
 
    train_loader = torch.utils.data.DataLoader(
    full_dataset, batch_size=len(full_dataset),
     num_workers=2
)




    return (full_dataset, test_dataset.classes)




data_loader, classes = get_dataset(data_dir = './data/imagenet')

RuntimeError: The archive ILSVRC2012_devkit_t12.tar.gz is not present in the root directory or is corrupted. You need to download it externally and place it in ./data/imagenet.